In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.wkt import loads

# 신천 교량별 15분 영향권 데이터

In [2]:
#데이터 읽어오기
df = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/02population/bridge_polygon.csv')
df = df[['name', 'geometry']]
df.columns = ['bridge', 'geometry']

#geometry 열 polygon type으로 생성
def load_valid(geo):
    try:
        return loads(geo)
    except:
        return np.nan
df.geometry = df.geometry.apply(load_valid)

gdf_sc = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:4326')
gdf_sc.head()

,bridge,geometry
0,가창,"POLYGON ((128.58400 35.89977, 128.58291 35.900..."
1,경대칠성,"POLYGON ((128.60581 35.86899, 128.60210 35.869..."
2,대봉희망,"POLYGON ((128.61472 35.84350, 128.61354 35.843..."
3,도청경대,"POLYGON ((128.59590 35.88019, 128.59317 35.883..."
4,동신수성,"POLYGON ((128.61729 35.85601, 128.60014 35.859..."


# 네이버 리뷰 데이터
관광지 정보 데이터와 리뷰 데이터가 분리되어있어  
관광지 정보 데이터에서 신천 영향권 내의 관광지 id 추출 후  
리뷰 데이터에서 해당 id의 리뷰만 추출

In [3]:
df_nSight = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/rawdata/31. 대구관광리뷰데이터/네이버_대구관광_장소데이터(utf-8).csv')
df_nSight.head()

,Unnamed: 0,ID,NAME,CATEGORY,ADDRESS_2,ADDRESS_1,LNG,LAT,REVIEW_COUNT,STAR_MEAN
0,0,1710520111,호텔리베,호텔,태평로2가 1-51,대구 중구 태평로2가,128.592952,35.875725,14,3.93
1,1,36139603,젠모텔,모텔,태평로2가 37-7,대구 중구 태평로2가,128.593098,35.875990,24,3.63
2,2,15970709,사보이 호텔,모텔,대안동 80-1 대구 동성로 사보이호텔,대구 중구 대안동,128.591106,35.872625,17,3.68
3,3,11622388,경상감영공원,테마공원,포정동 21,대구 중구 포정동,128.592229,35.872176,10,3.95
4,4,917513103,더한옥,전통숙소,북내동 4-1,대구 중구 북내동,128.590150,35.873083,53,3.49


In [4]:
df_nReview = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/rawdata/31. 대구관광리뷰데이터/네이버_대구관광_리뷰데이터(utf-8).csv')
df_nReview.head()

,Unnamed: 0,ID,STAR,REVIEW,N_VISIT,WRITE_DATE
0,0,1996982038,5.0,서울에서 내려오는 지인을 위해 예약했는데 같은지역의주민인 내가 가보니 찾기도 쉽고 ...,1번째 방문,2019.07.23
1,1,1996982038,4.0,저렴하게 잘 이용하고 왔습니다. 객실 깔끔하고 주택가에 위치해있어서 참 조용하니 좋...,1번째 방문,2018.03.02
2,2,1996982038,4.0,"사장님도 친절하시고, 편안하게 쉴 수 있었습니다.",1번째 방문,2021.05.07
3,3,1996982038,0.5,청소상태 너무 엉망이예요 걸을때마다 양말에 붙어니던 머리카락 덩어리하며 침대밑에 굴...,1번째 방문,2020.10.01
4,4,1996982038,5.0,깔끔한 숙소입니다. 잘 묵고 갑니다,1번째 방문,2020.07.05


In [9]:
#관광지 데이터 gdf로 만들기
df_nSight['geometry'] = df_nSight.apply(lambda row : Point([row['LNG'], row['LAT']]), axis=1)
gdf_nSight = gpd.GeoDataFrame(df_nSight, geometry='geometry', crs='epsg:4326')

#폴리곤 안에 있는 관광지만 추출
scSight = gpd.sjoin(gdf_sc, gdf_nSight)
scSight = scSight[['bridge', 'ID', 'NAME', 'CATEGORY', 'LNG', 'LAT']]

#ID기준 중복 제거
scSight = scSight.drop_duplicates('ID')

#호텔, 모텔, 게하 제거
con = scSight['CATEGORY'].isin(['호텔', '모텔', '게스트하우스'])
scSight = scSight[~con].reset_index(drop=True)

scSight.head()

c:\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,bridge,ID,NAME,CATEGORY,LNG,LAT
0,동신수성,1645037825,김광석스토리하우스,지역명소,128.605635,35.858978
1,두산용두,13470887,수성유원지,유원지,128.617942,35.829220
2,상동두산,13072214,아르떼 수성랜드,테마파크,128.613369,35.828590
3,상동두산,450414130,공룡공원,테마공원,128.603395,35.829925
4,성북도청,1295629728,아이니멀티플렉스,"관람,체험",128.589790,35.885226


In [11]:
#리뷰데이터와 머지
scSight = scSight.merge(df_nReview)
scSight = scSight[['bridge', 'NAME', 'WRITE_DATE', 'REVIEW', 'STAR', 'LNG', 'LAT']]

#df 양식 맞춰주기
scSight.columns = ['bridge', 'name', 'date', 'reviews', 'stars', 'lng', 'lat']
scSight['date'] = pd.to_datetime(scSight['date'], format="%Y.%m.%d").dt.strftime('%Y-%m')
scSight['stars'] = scSight['stars'].astype(int)
scSight = scSight.dropna()

scSight.head()

,bridge,name,date,reviews,stars,lng,lat
0,동신수성,김광석스토리하우스,2021-05,김광석을 좋아해서 와 봤어요. 좋네요~,5,128.605635,35.858978
1,동신수성,김광석스토리하우스,2021-05,김광석거리를 간다면 꼭 들려야 할 것 같아요 청음도 할 수 있고 김광석 이라는 가수...,5,128.605635,35.858978
2,동신수성,김광석스토리하우스,2021-04,좋아요~~~^^,4,128.605635,35.858978
3,동신수성,김광석스토리하우스,2021-05,김광석거리를 간다면 꼭 들려야 할 것 같아요 청음도 할 수 있고 김광석 이라는 가수...,5,128.605635,35.858978
4,동신수성,김광석스토리하우스,2021-04,친절한 안내 맘에 들었어요,5,128.605635,35.858978


In [12]:
scSight.to_csv('31naver_new.csv')

In [13]:
scSight.name.unique()

array(['김광석스토리하우스', '아이니멀티플렉스', '수성파크랜드'], dtype=object)